ATTENTION : this second step takes the database cleant and created at the end of this step [step1](https://www.kaggle.com/leticehs/nlp-analysis-part1-dataset-cut)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

First of all, I cut the dataset linked to this challenge in this notebook, https://www.kaggle.com/leticehs/nlp-analysis-part1-dataset-cut
Check it out first, as we will now use the dataset with cleant dates and only english tweets, below.

applying deep learning on twitter’s sentiment analysis as below

*   Prepare Data - download and prepare 21 millions tweets dat - check part 1 as linked above
*   Perform Pre-processing - use NLP techniques to pre-process the tweets => this is the notebook


Later on NLP3 to 5, I will describe the steps below, until topic clustering and prediction
*   Discover Important Words - use tf-idf and counting to find important words
*   Convert Word Representation - download word2vec and apply to the important word

*   Train Model - use keras to build and train a deep neural network model

*   Evaluate Model - measure the accuracy of the predictive model, and suggest further improvements


In [ ]:
!pip install contractions
from time import time
import pandas as pd
import numpy as np
import re
import sys
import csv
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import casual

import itertools
import datetime

import pprint
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# Reading the dataset with no columns titles and with latin encoding 
df = pd.read_csv('/kaggle/input/bitcoin-tweets-20160101-to-20190329/tweetsENdates.csv')
df.sample(3)


In [ ]:
# Checking if there is any missing value and datatype 
df.info()

In [ ]:
# checking for null values, if any
df.isnull().sum()

In [ ]:
#ditching all row when text is null, as need text for analysis
df.dropna(how='any', inplace=True)
df.sample(5)

In [ ]:
#setting column width wider to read text 
pd.set_option('display.max_colwidth',700)

In [ ]:
# Ommiting every column except for the text and the date, as we won't need any of the other information
df = df[['date','text']]
df.sample(3)

1 - Is dataset unbalanced ? do we need to split according to a criteria and take sample of those? splitting, skimming and concatening?

1a/ CLEANING DATE

In [ ]:
#Create split by dates 
df.date.dtypes

In [ ]:

df.date = pd.to_datetime(df['date'], format = '%Y-%m-%d')
df.sample(3)

1b/ Checking if tweets DO PROMOTE with subscribe

In [ ]:
#can we get counts of tweets that contain Follow us or subscribe?
(df['text'].str.contains('subscribe|follow')).sum()

In [ ]:
#checking the few tweets that do NOT contain follow or subscribe
(~df['text'].str.contains('subscribe|follow')).sum()

1c/ CHecking if tweets can be CLASSIFIED by TAGS


In [ ]:
#checking balance of tweets having # or not
df['text'].str.contains('#').sum()

SOME VISUALIZATION

In [ ]:
df = df.sort_values(by='date')

In [ ]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

In [ ]:
df.groupby('year').sum()

In [ ]:
df.year.value_counts().loc[lambda x : x>10000] 

In [ ]:
# Visualization of news article count per year

plt.figure(figsize=(10,5))
plt.xlabel("Year")
plt.ylabel("Counts")

df.year.value_counts().plot(kind='bar')
plt.show()

In [ ]:
# Visualization of news publications names and article count in the dataset

import matplotlib.pyplot as plt

color_list = list('rgbkymc')  #red, green, blue, black, etc.
plt.figure(figsize=(10,7))

plt.xlabel("date")
plt.ylabel("counts")

df.month.value_counts().plot(kind='bar', color=color_list)

2 - skimming tokenize, lemmatize

2a / PART OF SPEECH CLEANING

In [ ]:
#expanding contractions
try:
  df['text1'] = df['text'].apply(lambda x: [contractions.fix(word) for word in x.split()])
except:
  df['text1'] = df['text']


In [ ]:
#extracting the URL to clean what can be analysed
def get_url(x):
  urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', x)
  
  if len(urls)==0:
    return ' '
  else:
    return urls

In [ ]:
df['url'] = df['text1'].apply(lambda x: get_url(x) )
df['url'] = [' '.join(map(str, l)) for l in df['url']]
df.sample(3)

In [ ]:
#extracting the Tags to be able to clearer
def tag(x):
  hashtag = set(part[1:] for part in x.split() if part.startswith('#'))
  
  if len(hashtag)==0:
    return ' '
  else:
    return hashtag

df['tags'] = df['text1'].apply(lambda x: tag(x) )
df['tags'] = [' '.join(map(str, l)) for l in df['tags']]
df.sample(3)

In [ ]:
#extracting the follow or subscribe info on a specific column
def prom(x):
  promoting = re.findall('subscribe | follow | promo', x)
  
  if len(promoting)==0:
    return ' '
  else:
    return promoting

df['promote'] = df['text1'].apply(lambda x: prom(x) )
df['promote'] = [' '.join(map(str, l)) for l in df['promote']]
df.sample(3)

3 - CLEAN TEXT

In [ ]:
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
import regex as re
import contractions
# Install spaCy (run in terminal/prompt)
import sys


In [ ]:
#!!!!! CLEANING PROCESS, TAKES 38-40 minutes...
#this command needs to be pushed before tokenization
# >>>> COULD ADD LINE FOR AUTOCORRECT ET REMPLACER LES EMOJI PAR CE QU ILS MEAN

import nltk
nltk.download('stopwords')

def process(text):

    #Convert to lower case
    text = text.lower()
    #Convert www.* or https?://* to URL
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text)
    #Remove @username 
    text = re.sub('@[^\s]+',' ',text)
    #Remove additional white spaces
    text = re.sub('[\s]+', ' ', text)
    #Replace #word with word
    text = re.sub(r'#([^\s]+)', r'\1', text)
    #trim
    text = text.strip('\'"')

    #remove punctuation
    for punctuation in string.punctuation: 
        text = text.replace(punctuation, ' ') 

    #words only
    text = ''.join([i for i in text if not i.isdigit()])

    #tokenize
    words = nltk.tokenize.casual_tokenize(text)

    #Remove stopwords
    stops = set(stopwords.words('english'))
    clean = [w for w in words if not w in stops]

    return ' '.join(clean)

df['clean'] = df['text1'].apply(lambda text: process(text))

df.sample(3, random_state = 3)

In [ ]:
df = df.reset_index()
df = df[['date', 'year','clean','url','tags','promote']]

In [ ]:
#export to new CSV as it crashes otherwise
df.to_csv('/content/drive/MyDrive/Colab Notebooks/NLP/tweetsClean.csv')

no need to stem or lemmatize, the words are so small, that it ends up just letters

STOP VISU - TESTING WORD CLOUD WITH WHAT WE HAVE

In [ ]:
pip install wordcloud

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
# Light pre-processing
stopwords = set(STOPWORDS)
# Define Word Cloud generation function
def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stopwords,
        max_words=1000,
        max_font_size=40, 
        scale=3,
        random_state=1 # chosen at random by flipping a coin; it was heads
    ).generate(str(data))
    fig = plt.figure(1, figsize=(12, 12))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)
    plt.imshow(wordcloud)
    plt.show()
wordworld_text = df['clean']
show_wordcloud (wordworld_text)

Visualization n-grams of most important words

In [ ]:
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
#Mono-grams - takes a while - 10 minutes

def plot_top_ngrams_barchart(text, n=1):

    new= text.str.split()
    new=new.values.tolist()
    corpus=[word for i in new for word in i]

    def _get_top_ngram(corpus, n=None):
        vec = CountVectorizer(ngram_range=(n, n)).fit(corpus)
        bag_of_words = vec.transform(corpus)
        sum_words = bag_of_words.sum(axis=0) 
        words_freq = [(word, sum_words[0, idx]) 
                      for word, idx in vec.vocabulary_.items()]
        words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
        return words_freq[:20]

    top_n_bigrams=_get_top_ngram(text,n)[:20]
    x,y=map(list,zip(*top_n_bigrams))
    plt.figure(figsize=(10,10))
    plt.xlabel("Monogram Frequency")
    plt.ylabel("Top 20 KKW in EN tweets")
    sns.barplot(x=y,y=x)


plot_top_ngrams_barchart(df['clean'],1)


In [ ]:
#Bi-grams - takes a while - 5 minutes

def plot_top_ngrams_barchart(text, n=2):

    new= text.str.split()
    new=new.values.tolist()
    corpus=[word for i in new for word in i]

    def _get_top_ngram(corpus, n=None):
        vec = CountVectorizer(ngram_range=(n, n)).fit(corpus)
        bag_of_words = vec.transform(corpus)
        sum_words = bag_of_words.sum(axis=0) 
        words_freq = [(word, sum_words[0, idx]) 
                      for word, idx in vec.vocabulary_.items()]
        words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
        return words_freq[:20]

    top_n_bigrams=_get_top_ngram(text,n)[:20]
    x,y=map(list,zip(*top_n_bigrams))
    plt.figure(figsize=(10,10))
    plt.xlabel("Bi-gram Frequency")
    plt.ylabel("Top 20 KW in EN tweets")
    sns.barplot(x=y,y=x)


plot_top_ngrams_barchart(df['clean'],2)


STOP NOW AND IT WILL BE FOLLOWED BY NOTEBOOK NLP 3